#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [ ]:

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import os
import pandas as pd
import sqlite3
import datetime as dt

LIMIT_LINE = 100

conn = sqlite3.connect('log.db')
cur = conn.cursor()

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8018), 
                            requestHandler=RequestHandler, allow_none=True)


# Добавление строки в лог
def add_log(sname):
    global LIMIT_LINE
    with open("log.csv", "r") as file1:
        line_list = file1.readlines()
    if len(line_list) >= LIMIT_LINE:
        date_save = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
        with open(f'logs/{date_save}.csv', "w") as file2:
            file2.write("".join(line_list))
        with open("log.csv", "w") as file3:
            pass
    f = open('log.csv','a')
    f.write(str(sname)+','+ dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +'\n')
    f.close()
    return True

server.register_function(add_log, 'add_log')

def return_logs(op, dl, dr):
    dt_format = "%Y-%m-%d %H:%M:%S"
    dl = dt.datetime.strptime(dl, dt_format)
    dr = dt.datetime.strptime(dr, dt_format)
    log_files_list = [f"logs/{i}" for i in os.listdir(path="./logs")[1:]]
    df = pd.read_csv("log.csv", names=["operation", "dtime"])
    for i in log_files_list:
        df = pd.concat([df, pd.read_csv(i, names=["operation", "dtime"])])
    df["dtime"] = pd.to_datetime(df["dtime"].apply(lambda x : dt.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")))
    x = [[i[0], str(i[1])] for i in df[(df.operation == op) & (df.dtime >= dl) & (df.dtime < dr)].values.tolist()]
    return x
server.register_function(return_logs, 'return_logs')

def add_log_db(name, time):
    now = dt.datetime.now()
    datetime_string = now.strftime("%Y-%m-%d %H:%M:%S")
    cur.execute(f"INSERT INTO logs VALUES ('{name}', {time}, '{datetime_string}')")
    conn.commit()
server.register_function(add_log_db, 'add_log_db')

def return_log_db(name, time_l, time_r, dt_l):
    res = cur.execute(f"SELECT * FROM logs WHERE name = '{name}' AND time BETWEEN {time_l} AND {time_r} AND datetime LIKE '{dt_l}%'")
    return list(res)
server.register_function(return_log_db, 'return_log_db')


print ("Listening on port 8018...")
server.serve_forever()


Listening on port 8018...


127.0.0.1 - - [23/Oct/2023 13:43:23] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 13:43:25] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 13:43:32] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 13:43:34] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 13:43:52] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 13:43:54] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 13:44:06] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2023 13:44:08] "POST /RPC2 HTTP/1.1" 200 -
